## Check limit classically

In [ ]:
import json
import os
fiel_new_vectors = "new_noun_vectors.json"
with open(fiel_new_vectors, 'r') as fp:
    new_noun_vectors = json.load(fp)

In [ ]:
if not os.path.exists("report/figure/results/"):
    os.makedirs("report/figure/results/")

In [ ]:
# start with file, register, smooth
import numpy as np
from words import sentences as sentences


triplets = [
    ["register", "smooth"],
    ["drip", "carry"],
    ["knock", "intercept"],
    ["bill", "accuse"]
]

verb_vectors = {}
true_sentences = {}
for triplet in triplets:

    
    nouns, verbs = set(), set()
    for sentence in sentences:
        _s = sentence.split(" ")
        if _s[0] in triplet:
            true_sentences[(_s[0],_s[1])] = int(_s[2][1])
            verbs.add(_s[0])
            nouns.add(_s[1])

    n_verbs = len(verbs)
    noun_list = list(nouns)
    verb_list = list(verbs)
    all_combinations = [(verb, noun) for verb in verb_list for noun in noun_list]
    plausabilities = [true_sentences.get(sent,0) for sent in all_combinations]
    S = np.array(plausabilities).reshape((n_verbs,len(nouns)))
    N = np.array([new_noun_vectors[noun] for noun in noun_list]).transpose()

    N_inv = np.linalg.pinv(N)
    V = np.dot(S,N_inv)
    v_calc = {verb: V[i]/np.linalg.norm(V[i]) for i,verb in enumerate(verbs)}
    verb_vectors = {**verb_vectors, **v_calc}

In [ ]:
verb_vectors

In [ ]:
loss = []
for sentence in sentences:
    _s = sentence.split(" ")
    true_result = int(_s[2][1])
    try:
        calc_result = np.abs(np.dot(new_noun_vectors[_s[1]],verb_vectors[_s[0]]))
        loss.append((true_result-calc_result)**2)
    except:
        pass
mse = np.mean(loss)
print("Baseline: ", mse)

In [ ]:
folder = "experiments/density_matrix_model"
evo_path = os.path.join(folder,"evo.json")
with open(evo_path) as json_file:
    evo = json.load(json_file)

In [ ]:
import matplotlib.pyplot as plt

dims = (10,7)
fig, ax = plt.subplots(figsize=dims)
plt.rcParams["font.size"] = "14"
plt.plot(range(len(evo)), evo, '-b', label='loss')
plt.plot(range(len(evo)), [mse]*len(evo), '--r', label='baseline')
plt.ylim([0,0.3])
plt.xlabel("Epochs")
plt.ylabel("MSE")
plt.legend(loc='upper right')
#plt.title("temp")
plt.savefig("./report/figure/results/spsa_density_matrix_fit.pdf")

## Calculate density matrices from quantum fits

In [ ]:
# load the best fit
with open(os.path.join(folder,"params.json"), 'r') as fp:
    quantum_params = json.load(fp)
print(quantum_params["register"])

In [ ]:
# load the quantum ansätze
from discopy.quantum import CX, Circuit, CRz, H, Ket, Rx, Rz, Ry, X, sqrt, C, SWAP, CRy, Bra
n_verb_params = 6


def verb_ansatz(p):
    return Ket(0,0) >> \
        Rx(p[0]) @ Rx(p[1]) >> \
        Ry(p[2]) @ Ry(p[3]) >> \
        Rz(p[4]) @ Rz(p[5]) >> \
        CX >> SWAP >> CX >> SWAP 
        
def noun_ansatz(arr):
    a1 = np.linalg.norm(arr[0:2])
    a2 = np.linalg.norm(arr[2:])
    phi1 = np.arccos(a1)/np.pi

    # fix issues with rotations
    rot1 = arr[0:2]/a1
    phi2_cos = np.arccos(rot1[0])/np.pi
    phi2_sin = np.arcsin(rot1[1])/np.pi
    if not np.sign(phi2_cos) == np.sign(phi2_sin):
        phi2_cos *= -1
    rot2 = arr[2: ]/a2
    phi3_cos = np.arccos(rot2[0])/np.pi
    phi3_sin = np.arcsin(rot2[1])/np.pi
    if not np.sign(phi3_cos) == np.sign(phi3_sin):
        phi3_cos *= -1

    return Ket(0,0) >> Ry(phi1) @ Circuit.id(1) >> CRy(phi3_cos) >> X @ Circuit.id(1) >> CRy(phi2_cos) >> X @ Circuit.id(1)

### start with "file"

In [ ]:
## quantum model
verb_states = {verb: verb_ansatz(quantum_params[verb]["p"]).eval().array.flatten() for verb in verb_vectors}
noun_states = {noun: noun_ansatz(quantum_params[noun]["p"]).eval().array.flatten() for noun in new_noun_vectors}

# purely classical model
#verb_states = verb_vectors
#noun_states = new_noun_vectors

In [ ]:
#classically calculated loss
loss = []
k=0
for sentence in sentences:
    _s = sentence.split(" ")
    true_result = int(_s[2][1])
    try:
        calc_result = np.abs(np.dot(noun_states[_s[1]],verb_states[_s[0]]))
        #print(sentence)
        #print(calc_result)
        loss.append((true_result-calc_result)**2)
    except:
        pass
    
mse = np.mean(loss)
#print(loss)
print("Classically calc MSE: ", mse)

In [ ]:
verb = verb_states["smooth"]
noun = noun_states["tooth"]
np.abs(np.dot(verb,noun))**2

In [ ]:
rho_file = 1/2 * np.outer(np.conj(verb_states["register"]),verb_states["register"]) + \
    1/2 * np.outer(np.conj(verb_states["smooth"]),verb_states["smooth"])
print(rho_file.round(3))

In [ ]:
from scipy.linalg import logm
def von_neumann_entropy(rho):
    return np.abs(np.trace(np.dot(rho, logm(rho))))

von_neumann_entropy(rho_file)

In [ ]:
rho_account = np.outer(np.conj(noun_states["account"]),noun_states["account"])
von_neumann_entropy(rho_account).round(5)
von_neumann_entropy(np.multiply(rho_account,rho_file))

In [ ]:
from words import noun_groups, pairs
from pprint import pprint

s_dict = {}
disamb = []
disamb_rand = []
for amb_verb in pairs:
    sense1, sense2 = pairs[amb_verb]
    rho = 1/2 * np.outer(verb_states[sense1],np.conj(verb_states[sense1])) + \
        1/2 * np.outer(verb_states[sense2],np.conj(verb_states[sense2]))
    s = von_neumann_entropy(rho)
    s_dict[amb_verb] = {"init": s, "disamb": {}, "rand": []}

    for noun in noun_groups[amb_verb]:
        rho_n = np.outer(noun_states[noun],np.conj(noun_states[noun]))
        rho_comp = np.multiply(rho_n,rho)
        rho_comp /= np.trace(rho_comp)
        s_comp = von_neumann_entropy(rho_comp)
        s_dict[amb_verb]["disamb"][noun] = s_comp
        disamb.append((s-s_comp)/s)

        i=0
        while i<10:
            # per word get some random vectors for disambiguation
            z = np.random.uniform(-1,1,size=(4,))#.view(np.complex128)
            z /= np.linalg.norm(z)

            l = []
            for noun in noun_groups[amb_verb]:
                l.append(np.abs(np.dot(z,noun_states[noun])))
            if any(i >= 0.7 for i in l):
                continue
            i+=1
            rho_rand = np.outer(z,np.conj(z))
            rho_comp_rand = np.multiply(rho_rand,rho)
            rho_comp_rand /= np.trace(rho_comp_rand)
            s_comp_rand = von_neumann_entropy(rho_comp_rand)
            s_dict[amb_verb]["rand"].append(von_neumann_entropy(rho_comp_rand))
            disamb_rand.append((s-s_comp_rand)/s)

pprint(s_dict)

In [ ]:
import seaborn as sns
import pandas as pd
sns.set_theme(style="whitegrid")
# create boxplots
n_list = ["context" for i in range(len(disamb))]
r_list = ["random" for i in range(len(disamb_rand))]
data = pd.DataFrame({"Disambigutation Power": disamb+disamb_rand, "type": n_list+r_list})

dims = (10,7)
fig, ax = plt.subplots(figsize=dims)
b = sns.boxplot(ax=ax, x="type", y="Disambigutation Power", data=data)
b.tick_params(labelsize=14)
b.set_xlabel("Type",fontsize=16)
b.set_ylabel("Disambigutation Power",fontsize=16)
ax.set(ylim=(-0.2, 1));  

In [ ]:
print(np.median(disamb))
print(np.median(disamb_rand))
print(len(n_list))
print(len(r_list))

In [ ]:
from scipy.stats import ranksums

ranksums(disamb,disamb_rand)

## Similarity measure of disambiguated verbs

In [ ]:
from words import noun_groups, pairs
from pprint import pprint
import pandas as pd

save = {"word": [], "similarity": [], "score": []}
results = {amb_verb: [] for amb_verb in pairs}
for amb_verb in pairs:
    # construct density matrix of ambigious verb
    sense1, sense2 = pairs[amb_verb]
    rho = 1/2 * np.outer(verb_states[sense1],np.conj(verb_states[sense1])) + \
        1/2 * np.outer(verb_states[sense2],np.conj(verb_states[sense2]))

    for noun in noun_groups[amb_verb]:
        rho_n = np.outer(noun_states[noun],np.conj(noun_states[noun]))
        rho_comp = np.multiply(rho_n,rho)
        rho_comp /= np.trace(rho_comp)

        # calculate similarity to random word
        z = np.random.uniform(-1,1,size=(4,))
        z /= np.linalg.norm(z)
        rho_rand = np.matrix(np.outer(z,np.conj(z)))
        rho_comp_rand = np.multiply(rho_rand,rho)
        rho_comp_rand /= np.trace(rho_comp_rand)

        max_rand_sim = 0
        for sense in pairs[amb_verb]:

            rho_sense = np.matrix(np.outer(verb_states[sense],np.conj(verb_states[sense])))
            similarity = np.abs(np.trace(np.dot(rho_sense.getH(),rho_comp)))
            gtruth = true_sentences.get((sense,noun),0)
            
            save["word"].append(amb_verb)

            if gtruth == 0:
                save["similarity"].append("context")
                save["score"].append(1-similarity)
                results[amb_verb].append([gtruth, similarity])
            else:
                save["similarity"].append("context")
                save["score"].append(similarity)
                results[amb_verb].append([gtruth, similarity])

            similarity_rand = np.abs(np.trace(np.dot(rho_sense.getH(),rho_comp_rand)))
            max_rand_sim = max(max_rand_sim,similarity_rand)

        save["word"].append(amb_verb)
        save["similarity"].append("random")
        save["score"].append(max_rand_sim)
            
save_df = pd.DataFrame.from_dict(save)
save_df.head(15)

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt
sns.set(font_scale=1.5)
dims = (10,7)
fig, ax = plt.subplots(figsize=dims)


sns.set_theme(style="whitegrid")
tips = sns.load_dataset("tips")
sns.boxplot(ax=ax, x="word", y="score", hue="similarity",
                 data=save_df, palette="Set3")
plt.legend(prop={"size":15});

In [ ]:
# check for difference in distribution
for amb_verb in results:
    idx1 = (save_df.word == amb_verb) & (save_df.similarity=="context")
    idx2 = (save_df.word == amb_verb) & (save_df.similarity=="random")
    print(amb_verb,ranksums(save_df[idx1].score,save_df[idx2].score))